In [1]:
%pip install plotly pandas

import pandas as pd
import plotly.graph_objects as go

# Load the dataset
df_oil = pd.read_csv('../../../data/outputs/filtered_oil_data.csv')

# List the companies in the dataset
companies = df_oil['Company'].unique()

# Available columns (metrics) to plot
available_columns = ['Adj. Close', 'Volume', 'Open', 'Close', 'High', 'Low']


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Available columns (metrics) to plot
available_columns = ['Adj Close', 'Volume', 'Open', 'High', 'Low']

# Create an empty figure object
fig = go.Figure()

# Add traces for all three companies (initially set to show 'Adj Close')
for company in companies:
    company_data = df_oil[df_oil['Company'] == company]
    fig.add_trace(go.Scatter(x=company_data['Date'], y=company_data['Adj Close'], mode='lines', name=company))

# Create dropdowns for company selection and metric selection with adjusted labels
fig.update_layout(
    title="Oil Data: Adjusted Close Price",
    xaxis_title="Date",
    yaxis_title="Value",
    updatemenus=[
        # Dropdown for selecting company
        dict(
            buttons=[
                dict(
                    label="All Companies",
                    method="update",
                    args=[{"visible": [True, True, True]},
                          {"title": "Oil Data: Adjusted Close Price (All Companies)"}]
                )
            ] + [
                dict(
                    label=company,
                    method="update",
                    args=[{"visible": [company == comp for comp in companies]},
                          {"title": f"Oil Data: Adjusted Close Price ({company})"}]
                ) for company in companies
            ],
            direction="down",
            showactive=True,
            pad={"r": 10},  # Add padding to the right of the dropdown
            x=0.35,  # Adjust position to fit inline with label
            xanchor="left",
            y=1.15,
            yanchor="top",
        ),
        # Dropdown for selecting metric (e.g., Adj Close, Volume, etc.)
        dict(
            buttons=[dict(
                label=metric,
                method="update",
                args=[{"y": [df_oil[df_oil['Company'] == company][metric] for company in companies]},
                      {"title": f"Oil Data: {metric}"}]
            ) for metric in available_columns],
            direction="down",
            showactive=True,
            pad={"r": 10},  # Add padding to the right of the dropdown
            x=0.75,  # Adjust position to fit inline with label
            xanchor="left",
            y=1.15,
            yanchor="top",
        )
    ]
)

# Add labels for dropdowns (Company: DROPDOWN  Metric: DROPDOWN)
fig.add_annotation(x=0.25, y=1.18, xref='paper', yref='paper', showarrow=False, text="Company:", font=dict(size=12))
fig.add_annotation(x=0.65, y=1.18, xref='paper', yref='paper', showarrow=False, text="Metric:", font=dict(size=12))

# Show the figure
fig.show()


In [3]:
# Save the oil plot as an HTML file
fig.write_html('oil_plotly.html')
